We choose one of the pipelines that TPOT used and then use SigOpt to optimize the hyperparametrs. We compare the resulting performance to TPOT.

# Initialize

In [1]:
import dill as pickle
import numpy as np
from sklearn import metrics
from tpot import TPOTRegressor
from gaspy.utils import vasp_settings_to_str
from gaspy_regress.regressor import GASpyRegressor

# Fit TPOT

## Settings

In [ ]:
fit_blocks = [('CO',)]
dev_size = None
dim_red = 'pca'
tpot_verbosity = 2
n_jobs = 16
random_state = None
VASP_SETTINGS = vasp_settings_to_str({'gga': 'RP',
                                      'pp_version': '5.4',
                                      'encut': 350})
model_name = 'TPOT'
features = ['coordatoms_chemfp0', 'neighbors_chemfp0']
responses = ['energy']
blocks = ['adsorbate']

## Standard
Let's run a single TPOT on 90% of our data and see what the train/test errors are.

In [ ]:
train_size = 0.9
tpot_generations = 1
tpot_population = 16
tpot_offspring = 16

# Fit
tpot = TPOTRegressor(generations=tpot_generations,
                     population_size=tpot_population,
                     offspring_size=tpot_offspring,
                     verbosity=tpot_verbosity,
                     random_state=random_state,
                     n_jobs=n_jobs)
model = GASpyRegressor(features=features, responses=responses,
                       blocks=blocks, vasp_settings=VASP_SETTINGS,
                       train_size=train_size, dev_size=dev_size,
                       dim_red=dim_red)
model.fit_tpot(tpot, model_name=model_name, blocks=fit_blocks)

# Save
with open('model.pkl', 'w') as file_handle:
    pickle.dump(model, file_handle)

In [15]:
# Load
with open('standard_model.pkl', 'r') as file_handle:
    model = pickle.load(file_handle)

In [16]:
# Calculate errors and metrics
errors = model.residuals[('CO',)]['test']
zeros = np.zeros(errors.shape[0])
rmse = np.sqrt(metrics.mean_squared_error(zeros, errors))
mae = metrics.mean_absolute_error(zeros, errors)
mad = metrics.median_absolute_error(zeros, errors)

# Report pipeline and metrics
for step in model.models[('CO',)].steps:
    print(step)
print('    RMSE = %.2f' % rmse)
print('    RMSE = %.2f' % mae)
print('    RMSE = %.2f' % mad)

('stackingestimator', StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8, max_features=1.0,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=16,
             min_samples_split=4, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=0.65, verbose=0, warm_start=False)))
('randomforestregressor', RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features=0.3, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=7, min_samples_split=7,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))
    RMSE = 0.32
    RMSE = 0.18
    RMSE = 0.10


## Extended
Let's run a single TPOT on all of our data and let it go for awhile. We'll use the final pipeline setup as the skeleton for SigOpt to start with. Note that since this takes awhile, we actualy ran this via Python/Shell/SLURM, not Jupyter. The CV score that TPOT reported for this one was 0.109350437412, which does not seem much better than the 0.116494047003 we got with only 1 generation and population of 16

In [ ]:
train_size = 1.
tpot_generations = 25
tpot_population = 64
tpot_offspring = 64

# Fit
tpot = TPOTRegressor(generations=tpot_generations,
                     population_size=tpot_population,
                     offspring_size=tpot_offspring,
                     verbosity=tpot_verbosity,
                     random_state=random_state,
                     n_jobs=n_jobs)
model = GASpyRegressor(features=features, responses=responses,
                       blocks=blocks, vasp_settings=VASP_SETTINGS,
                       train_size=train_size, dev_size=dev_size,
                       dim_red=dim_red)
model.fit_tpot(tpot, model_name=model_name, blocks=fit_blocks)

# Save
with open('model.pkl', 'w') as file_handle:
    pickle.dump(model, file_handle)

In [ ]:
# Load
with open('model.pkl', 'r') as file_handle:
    model = pickle.load(file_handle)

In [ ]:
# Report pipeline
for step in model.models[('CO',)].steps:
    print(step)

# SigOpt

In [2]:
from sigopt_sklearn.search import SigOptSearchCV
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import FastICA
from tpot.builtins.stacking_estimator import StackingEstimator
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor

## Pre-work

In [3]:
import tqdm
from gaspy import defaults, gasdb, utils
from gaspy_regress.preprocessor import GASpyPreprocessor
from pymatgen.matproj.rest import MPRester
from sklearn.model_selection import train_test_split

### Pull data

In [4]:
# Pull all of the documents and sort them by date
fingerprints = defaults.fingerprints(simulated=True)
fingerprints['symbols'] = '$atoms.chemical_symbols'  # needed for preprocessor
docs = gasdb.get_docs(fingerprints=fingerprints, **defaults.doc_filters())
docs = sorted(docs, key=lambda doc: doc['adslab_calculation_date'])

# Let's just look at CO for this run
docs = [doc for doc in docs if 'CO' in doc['adsorbates']]

# Initialize a dictionary whose keys are the mpids we've studied
# and whose values are their compositions. This can be used to filter
# the documents
mpids = set(doc['mpid'] for doc in docs)
compositions_by_mpid = dict.fromkeys(mpids)
# Open up MP connection
mp_key = utils.read_rc('matproj_api_key')
with MPRester(mp_key) as mp_db:
    # Populate that dictionary, `compositions_by_mpid`
    for mpid in tqdm.tqdm(mpids):
        entry = mp_db.get_entry_by_material_id({'task_ids': mpid})
        comp = set(entry.as_dict()['composition'].keys())
        compositions_by_mpid[mpid] = set(comp)

# Define the elements to exclude from our study
excluded_elements = set(['Ca', 'Na', 'Nb', 'S', 'Se'])
# Use the `compositions_by_mpid` dictionary to identify the mpid numbers to exclude.
mpids_to_exclude = []
for mpid, comp in compositions_by_mpid.iteritems():
    if excluded_elements.intersection(comp):
        mpids_to_exclude.append(mpid)
mpids_to_exclude = set(mpids_to_exclude)

# Finally, reconstruct docs to exclude the elements
docs = [doc for doc in docs if doc['mpid'] not in mpids_to_exclude]
# Add a key to the docs that we're probably going to need
for i, doc in enumerate(docs):
    ads = doc['adsorbates'][0]
    docs[i]['adsorbate'] = ads

Starting to pull documents...


52328it [00:01, 35199.95it/s]
/project/projectdirs/m2755/GASpy_conda/lib/python2.7/site-packages/pymatgen-2017.7.4-py2.7-linux-x86_64.egg/pymatgen/ext/matproj.py:106: UserWarning:

If you query for structure data encoded using MP's Structure Notation Language (SNL) format and you use `mp_decode=True` (the default) for MPRester queries, you should install dependencies via `pip install pymatgen[matproj.snl]`.

100%|██████████| 1312/1312 [03:03<00:00,  7.27it/s]


### Fit the preprocessor

In [6]:
# Settings
train_size = 0.9
features = ['coordatoms_chemfp0', 'neighbors_chemfp0']
dim_red = 'pca'

# Split into train/test
docs_train, docs_test = train_test_split(docs, train_size=train_size)
y = np.array([doc['energy'] for doc in docs])
y_train = np.array([doc['energy'] for doc in docs_train])
y_test = np.array([doc['energy'] for doc in docs_test])

# Make the preprocessor and then use it to get the x train/test data
pp = GASpyPreprocessor(docs=docs_train, features=features, dim_red=dim_red)
x = pp.transform(docs)
x_train = pp.transform(docs_train)
x_test = pp.transform(docs_test)

/project/projectdirs/m2755/GASpy_conda/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



## The "simple" pipeline

In [ ]:
#gbr = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
#                                learning_rate=0.1, loss='ls', max_depth=8,
#                                max_features=1.0, max_leaf_nodes=None,
#                                min_impurity_decrease=0.0, min_impurity_split=None,
#                                min_samples_leaf=16, min_samples_split=4,
#                                min_weight_fraction_leaf=0.0, n_estimators=100,
#                                presort='auto', random_state=None, subsample=0.65,
#                                verbose=0, warm_start=False)
#stacking_est = StackingEstimator(estimator=gbr)
## Step 2
#rfr = RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
#                            max_features=0.3, max_leaf_nodes=None,
#                            min_impurity_decrease=0.0, min_impurity_split=None,
#                            min_samples_leaf=7, min_samples_split=7,
#                            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
#                            oob_score=False, random_state=None, verbose=0, warm_start=False)
## Create pipeline
#pipeline = make_pipeline(stacking_est, rfr)

## The "complicated" pipeline

In [ ]:
## Parameters. The ranges were taken from tpot.config.regressor
#sigopt_params = {'ica_tol': (0.0, 1.01),
#                 'elastic_net_l1': (0.0, 1.01),
#                 'elastic_net_tol': (1e-5, 1e-1),
#                 'extra_trees_max_features': (0.05, 1.01),
#                 'extra_trees_min_samples_split': (2, 21),
#                 'extra_trees_min_samples_leaf': (1, 21),
#                 'extra_trees_bootstrap': [True, False] }
#
## First step in the pipeline
## The big pipeline uses some function that I don't really know about.
## Let's simply pull the function out and use it, eh?
#func = model.models[('CO',)].steps[0][1].get_params()['functiontransformer'].func
#func_trans = FunctionTransformer(accept_sparse=False, func=func,
#                                 inv_kw_args=None, inverse_func=None,
#                                 kw_args=None, pass_y='deprecated',
#                                 validate=True)
#fast_ica = FastICA(algorithm='parallel', fun='logcosh', fun_args=None,
#                   max_iter=200, n_components=None, random_state=None,
#                   tol=0.05, w_init=None, whiten=True)
#feat_union = FeatureUnion(n_jobs=1,
#                          transformer_list=[('functiontransformer', func_trans),
#                                            ('fastica', fast_ica)],
#                          transformer_weights=None)
#
## Second step
#elastic_net = ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001,
#                           fit_intercept=True, l1_ratio=0.4, max_iter=1000,
#                           n_alphas=100, n_jobs=1, normalize=False,
#                           positive=False, precompute='auto', random_state=None,
#                           selection='cyclic', tol=0.001, verbose=0)
#stacking_est = StackingEstimator(estimator=elastic_net)
#
## Third step
#extra_trees = ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
#                                  max_features=1.0, max_leaf_nodes=None,
#                                  min_impurity_decrease=0.0, min_impurity_split=None,
#                                  min_samples_leaf=1, min_samples_split=12,
#                                  min_weight_fraction_leaf=0.0, n_estimators=100,
#                                  n_jobs=1, oob_score=False, random_state=None,
#                                  verbose=0, warm_start=False)
#
## Make the pipeline
#pipeline = make_pipeline(feat_union, stacking_est, extra_trees)

## Just a random forest
It turns out the SigOpt API isn't really configured to optimize pipelines quite yet... woops. Let's just use a single layer RandomForest, which showed up once in awhile in the convergence testing.

In [17]:
# We got these parameter ranges from tpot.config.regressor
RF = RandomForestRegressor()
RF_parameters = {'max_features': (0.05, 1.01),
                 'min_samples_split': (2, 21),
                 'min_samples_leaf': (1, 21)}

sigopt = SigOptSearchCV(RF, RF_parameters, cv=5,
                        client_token='GUEBFWNSXCGPZQZFOESYAIYUWSBSZFKJZPZJFWZNBKCTSTHG',
                        n_sug=2, n_jobs=16, n_iter=100, verbose=2,
                        scoring='neg_mean_absolute_error')

sigopt.fit(x_train, y_train)

Creating SigOpt experiment:  RandomForestRegressor (sklearn)
Experiment progress available at : https://sigopt.com/experiment/47114
Evaluating params :  [{'max_features': 0.32708070807579925, 'min_samples_split': 14, 'min_samples_leaf': 3}, {'max_features': 0.32708070807579925, 'min_samples_split': 14, 'min_samples_leaf': 3}, {'max_features': 0.32708070807579925, 'min_samples_split': 14, 'min_samples_leaf': 3}, {'max_features': 0.32708070807579925, 'min_samples_split': 14, 'min_samples_leaf': 3}, {'max_features': 0.32708070807579925, 'min_samples_split': 14, 'min_samples_leaf': 3}, {'max_features': 0.8060371502412993, 'min_samples_split': 11, 'min_samples_leaf': 18}, {'max_features': 0.8060371502412993, 'min_samples_split': 11, 'min_samples_leaf': 18}, {'max_features': 0.8060371502412993, 'min_samples_split': 11, 'min_samples_leaf': 18}, {'max_features': 0.8060371502412993, 'min_samples_split': 11, 'min_samples_leaf': 18}, {'max_features': 0.8060371502412993, 'min_samples_split': 11, '

[Parallel(n_jobs=16)]: Done   3 out of  10 | elapsed:    0.8s remaining:    1.8s


[CV]  max_features=0.806037150241, min_samples_split=11, min_samples_leaf=18, total=   1.2s
[CV]  max_features=0.806037150241, min_samples_split=11, min_samples_leaf=18, total=   1.2s
[CV]  max_features=0.806037150241, min_samples_split=11, min_samples_leaf=18, total=   1.2s
[CV]  max_features=0.806037150241, min_samples_split=11, min_samples_leaf=18, total=   1.2s
[CV]  max_features=0.806037150241, min_samples_split=11, min_samples_leaf=18, total=   1.2s


[Parallel(n_jobs=16)]: Done  10 out of  10 | elapsed:    1.3s finished


Evaluating params :  [{'max_features': 0.07440383037050452, 'min_samples_split': 18, 'min_samples_leaf': 14}, {'max_features': 0.07440383037050452, 'min_samples_split': 18, 'min_samples_leaf': 14}, {'max_features': 0.07440383037050452, 'min_samples_split': 18, 'min_samples_leaf': 14}, {'max_features': 0.07440383037050452, 'min_samples_split': 18, 'min_samples_leaf': 14}, {'max_features': 0.07440383037050452, 'min_samples_split': 18, 'min_samples_leaf': 14}, {'max_features': 0.6998616721279795, 'min_samples_split': 5, 'min_samples_leaf': 8}, {'max_features': 0.6998616721279795, 'min_samples_split': 5, 'min_samples_leaf': 8}, {'max_features': 0.6998616721279795, 'min_samples_split': 5, 'min_samples_leaf': 8}, {'max_features': 0.6998616721279795, 'min_samples_split': 5, 'min_samples_leaf': 8}, {'max_features': 0.6998616721279795, 'min_samples_split': 5, 'min_samples_leaf': 8}]
[CV] max_features=0.0744038303705, min_samples_split=18, min_samples_leaf=14 
[CV] max_features=0.0744038303705, 

...

Evaluating params :  [{'max_features': 0.24433635932449455, 'min_samples_split': 2, 'min_samples_leaf': 16}, {'max_features': 0.24433635932449455, 'min_samples_split': 2, 'min_samples_leaf': 16}, {'max_features': 0.24433635932449455, 'min_samples_split': 2, 'min_samples_leaf': 16}, {'max_features': 0.24433635932449455, 'min_samples_split': 2, 'min_samples_leaf': 16}, {'max_features': 0.24433635932449455, 'min_samples_split': 2, 'min_samples_leaf': 16}, {'max_features': 0.7267323408530705, 'min_samples_split': 6, 'min_samples_leaf': 14}, {'max_features': 0.7267323408530705, 'min_samples_split': 6, 'min_samples_leaf': 14}, {'max_features': 0.7267323408530705, 'min_samples_split': 6, 'min_samples_leaf': 14}, {'max_features': 0.7267323408530705, 'min_samples_split': 6, 'min_samples_leaf': 14}, {'max_features': 0.7267323408530705, 'min_samples_split': 6, 'min_samples_leaf': 14}]
[CV] max_features=0.244336359324, min_samples_split=2, min_samples_leaf=16 
[CV] max_features=0.244336359324, min

[Parallel(n_jobs=16)]: Done   3 out of  10 | elapsed:    0.5s remaining:    1.1s


[CV]  max_features=0.726732340853, min_samples_split=6, min_samples_leaf=14, total=   1.1s
[CV]  max_features=0.726732340853, min_samples_split=6, min_samples_leaf=14, total=   1.1s
[CV]  max_features=0.726732340853, min_samples_split=6, min_samples_leaf=14, total=   1.1s
[CV]  max_features=0.726732340853, min_samples_split=6, min_samples_leaf=14, total=   1.1s
[CV]  max_features=0.726732340853, min_samples_split=6, min_samples_leaf=14, total=   1.1s


[Parallel(n_jobs=16)]: Done  10 out of  10 | elapsed:    1.2s finished


SigOptSearchCV(client_token=None, cv=5, cv_timeout=None, error_score='raise',
        estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
        fit_params=None, iid=True, n_iter=100, n_jobs=16, n_sug=2,
        opt_timeout=None,
        param_domains={'max_features': (0.05, 1.01), 'min_samples_split': (2, 21), 'min_samples_leaf': (1, 21)},
        pre_dispatch='2*n_jobs', refit=True,
        scoring='neg_mean_absolute_error',
        sigopt_connection=<sigopt.interface.Connection object at 0x2ae90aa08110>,
        verbose=2)

In [18]:
# Calculate errors and metrics
y_true = y_test
y_pred = sigopt.predict(x_test)
rmse = np.sqrt(metrics.mean_squared_error(y_true, y_pred))
mae = metrics.mean_absolute_error(y_true, y_pred)
mad = metrics.median_absolute_error(y_true, y_pred)

# Report pipeline and metrics
print('RMSE = %.2f' % rmse)
print('RMSE = %.2f' % mae)
print('RMSE = %.2f' % mad)

RMSE = 0.33
RMSE = 0.19
RMSE = 0.10
